In [9]:
import face_recognition
import cv2
import numpy as np
import wget
video_capture = cv2.VideoCapture(0)

fgbg = cv2.createBackgroundSubtractorMOG2(300, 400, True)
font = cv2.FONT_HERSHEY_SIMPLEX

frameCount = 0

shubham_image = face_recognition.load_image_file(wget.download("https://i.imgur.com/a1HmdVp.jpg"))
shubham_face_encoding = face_recognition.face_encodings(shubham_image)[0]


known_face_encodings = [
    shubham_face_encoding,
]
known_face_names = [
    "Shubham ", 
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    ret, frame = video_capture.read()
    prvs = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame)
    hsv[...,1] = 255
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        found=0
        for face_encoding in face_encodings:
            found=1
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            face_names.append(name)
    process_this_frame = not process_this_frame

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        cv2.putText(frame, name, (left*4+6,top*4), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    frameCount += 1   
    resizedFrame = cv2.resize(frame, (0, 0), fx=0.50, fy=0.50) 
    fgmask = fgbg.apply(resizedFrame) 
    count = np.count_nonzero(fgmask)  
    if (frameCount > 10 and count > 2700):
        cv2.putText(resizedFrame, name+' cheating', (10, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
        count=0
    cv2.imshow('Frame', resizedFrame) # actual footage
    #cv2.imshow('Mask', fgmask)     # pixel changes
    if cv2.waitKey(1) & 0xFF == 27:
        break

video_capture.release()
cv2.destroyAllWindows()


In [8]:
ord('q')

113